# ASL Neural Network Pipeline Notebook

This notebook contains all the steps necessary to train a neural network for the ASL Neural Network App project located at [this repository](https://github.com/TWilliamsA7/asl-neural-app/tree/main). Utility functions can also be found in the above repository under the src directory.

1. Setup: Configuration & Authentication
2. Environment: Initialization & Imports
3. Data: Acquisition & Preprocessing
4. Data: Loading & Splitting
5. Model: Architecture
6. Model: Training
7. Model: Evaluation

## Setup: Configuration & Authenticatioon

This section of the notebook is for setting up the necessary authentication and configuration of the Colab environment

In [1]:
# Import necessary modules for setup

from google.colab import userdata, auth, files
import os
import sys

### Create github connection via colab variables

In [2]:
# Define repository details
USERNAME = "TWilliamsA7"
REPO_NAME = "asl-neural-app.git"
BRANCH_NAME = "main"

# Get PAT (Personal Access Token) stored in Colab Secrets
PAT = userdata.get("GITHUB_PAT")
if not PAT:
    raise ValueError("GITHUB_PAT secret not found!")

# Construct Authetnicated URL for accessing repositry
AUTHENTICATED_URL = f"https://{PAT}@github.com/{USERNAME}/{REPO_NAME}"
REPO_FOLDER = REPO_NAME.replace(".git", "")

# Set golabl Git configuration
!git config --global user.email "twilliamsa776@gmail.com"
!git config --global user.name "{USERNAME}"

print("Setup github connection and authenticated url successfully!")

Setup github connection and authenticated url successfully!


### Google Cloud Authentication

In [3]:
print("--- GCS Authentication ---")

auth.authenticate_user()

print("Google Cloud authentication complete.")

--- GCS Authentication ---
Google Cloud authentication complete.


## Environment: Initialization and Imports

### Clone Github Repository

In [4]:
# Clean up any existing clone (optional, but good for reliable restarts)
if os.path.isdir(REPO_FOLDER):
    print(f"Removing old {REPO_FOLDER} folder...")
    !rm -rf {REPO_FOLDER}

# Clone the repository using the authenticated URL
print(f"Cloning repository: {REPO_NAME}...")
!git clone {AUTHENTICATED_URL}

# Change directory into the cloned repository
%cd {REPO_FOLDER}
print(f"Current working directory: {os.getcwd()}")

Removing old asl-neural-app folder...
Cloning repository: asl-neural-app.git...
Cloning into 'asl-neural-app'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 97 (delta 42), reused 78 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 23.41 KiB | 4.68 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/asl-neural-app
Current working directory: /content/asl-neural-app


### Install Dependencies

- Includes manual inclusion of kaggle.json file

In [4]:
print("Upgrading pip, setuptools, and wheel...")
!pip install --upgrade pip setuptools wheel -q

print("Using preinstalled numpy and tensorflow dependencies")

print("Installing remaining project dependencies from requirements.txt...")
!pip install -r requirements.txt -q

print("Dependencies installed successfully.")

Upgrading pip, setuptools, and wheel...
Using preinstalled numpy and tensorflow dependencies
Installing remaining project dependencies from requirements.txt...
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Dependencies installed successfully.


### Setup .Kaggle Directory

In [6]:
# Check if the credentials file already exists in the expected location
if not os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
    print("Uploading kaggle.json file...")
    # This will open a dialog for you to select and upload your file
    uploaded = files.upload()

    # Check if the upload was successful
    if not uploaded:
        print("ERROR: kaggle.json was not uploaded.")
    else:
        # The uploaded file is now in the current working directory (/content/)
        # Proceed to move and secure it.

        # 2. Create the required directory
        !mkdir -p ~/.kaggle/

        # 3. Move the uploaded file into the correct directory
        # The key in the uploaded dictionary is the filename (kaggle.json)
        # We assume the user uploaded the file named 'kaggle.json'
        !mv kaggle.json ~/.kaggle/kaggle.json

        # 4. Set the correct permissions (CRITICAL)
        # Permissions MUST be 600 for security.
        !chmod 600 ~/.kaggle/kaggle.json

        print("Kaggle authentication file set up successfully!")
else:
    print("Kaggle credentials already found at ~/.kaggle/kaggle.json.")

# --- Verification Step ---
# Run a simple Kaggle command to test authentication
try:
    print("\nAttempting to list datasets (Verification)...")
    # This command uses the username/key from the now-configured kaggle.json
    !kaggle datasets list -s asl_alphabet | head -n 3
    print("\nSUCCESS: Kaggle API authenticated and is functional.")
except Exception as e:
    print(f"\nERROR: Verification failed. Please check the content of your kaggle.json file. Details: {e}")

Kaggle credentials already found at ~/.kaggle/kaggle.json.

Attempting to list datasets (Verification)...
ref                                      title                                 size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------  ------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
baoanhcr7/asl-alphabet                   ASL_alphabet                    1100898315  2021-11-06 20:56:29.557000            231          1  0.125            

SUCCESS: Kaggle API authenticated and is functional.


### Connect Src directory for access to utility functions

In [7]:
sys.path.append('src')
print("Setup Complete. Colab environment is ready.")

Setup Complete. Colab environment is ready.


## Data: Acquisition & Preprocessing

### Include necessary imports

In [8]:
import numpy as np
import cv2
import gc

# If earlier cells are not ran
import os
import sys

# Ensure src accessibility
sys.path.append('src')

# Import utility functions
from data_utils import extract_keypoints

### Setup directories and constants

In [9]:
KAGGLE_DATASET_ID = "grassknoted/asl-alphabet"
DESTINATION_PATH = "sample_data"
PROCESSED_OUTPUT_DIR = 'processed_data'
DATA_ROOT_FOLDER_NAME = 'asl_alphabet_train'

os.makedirs(DESTINATION_PATH, exist_ok=True)
os.makedirs(PROCESSED_OUTPUT_DIR, exist_ok=True)

### Download Data via Kaggle API

In [10]:
print(f"Downloading dataset: {KAGGLE_DATASET_ID}")
!kaggle datasets download -d {KAGGLE_DATASET_ID} -p {DESTINATION_PATH} --unzip

# Define the exact root path to the image subfolders (A, B, C, etc.)
DATA_ROOT = os.path.join(DESTINATION_PATH, DATA_ROOT_FOLDER_NAME, DATA_ROOT_FOLDER_NAME)
print(f"Image data root set to: {DATA_ROOT}")

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
 99% 1.01G/1.03G [00:21<00:00, 77.8MB/s]
100% 1.03G/1.03G [00:21<00:00, 51.9MB/s]
Image data root set to: sample_data/asl_alphabet_train/asl_alphabet_train


### Feature Extraction and Array Storage

In [ ]:
GCS_BUCKET_NAME = "gs://asl-keypoint-data-storage-2025"
GCS_DESTINATION_FOLDER = "processed_features_v1"

# 1. Get all unique class folder names and sort them alphabetically
class_names = sorted([d for d in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, d))])

# 2. Create the dictionary
label_map = {name: i for i, name in enumerate(class_names)}

FEATURE_OUTPUT_DIR = os.path.join('processed_data', 'class_splits')
os.makedirs(FEATURE_OUTPUT_DIR, exist_ok=True) # Ensure the directory exists

def create_and_save_features():
    # List to hold file paths of NPY files for later concatenation
    all_class_files = []

    # Iterate through all class folders
    for class_name in class_names:
        class_path = os.path.join(DATA_ROOT, class_name)
        label_index = label_map[class_name]

        print(f"Processing Class: {class_name} (Label: {label_index})")

        # --- Memory-Saving Block ---
        class_keypoints = []
        class_images = []
        class_labels = []

        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)

            # Use the imported modular function
            keypoints, resized_img = extract_keypoints(image_path)

            if keypoints is not None:
                class_keypoints.append(keypoints)
                class_images.append(resized_img)
                class_labels.append(label_index)

        # 3. Convert and Save (The memory-intensive part, done one class at a time)
        X_key_class = np.array(class_keypoints, dtype=np.float32)
        X_cnn_class = np.array(class_images, dtype=np.float32)
        y_class = np.array(class_labels, dtype=np.int32)

        # 4. Save to Disk
        # Use a temporary name for each class file
        key_file = os.path.join(FEATURE_OUTPUT_DIR, f'keypoints_{class_name}.npy')
        cnn_file = os.path.join(FEATURE_OUTPUT_DIR, f'cnn_{class_name}.npy')
        label_file = os.path.join(FEATURE_OUTPUT_DIR, f'labels_{class_name}.npy')

        np.save(key_file, X_key_class)
        np.save(cnn_file, X_cnn_class)
        np.save(label_file, y_class)
        all_class_files.append((key_file, cnn_file, label_file))

        print(f"Processed and saved {class_name}. Freeing memory...")

        # 5. Crucial: Delete objects and force garbage collection
        del X_key_class, X_cnn_class, y_class, class_keypoints, class_images, class_labels
        gc.collect()

    # --- Final Step: Concatenate all saved files ---
    # This part is still memory-intensive, but happens once at the end
    final_keypoints = np.concatenate([np.load(f[0]) for f in all_class_files])
    final_cnn_images = np.concatenate([np.load(f[1]) for f in all_class_files])
    final_labels = np.concatenate([np.load(f[2]) for f in all_class_files])

    # Save final concatenated arrays (as you did before)
    np.save(os.path.join(FEATURE_OUTPUT_DIR, 'X_keypoints.npy'), final_keypoints)
    np.save(os.path.join(FEATURE_OUTPUT_DIR, 'X_cnn_images.npy'), final_cnn_images)
    np.save(os.path.join(FEATURE_OUTPUT_DIR, 'y_labels.npy'), final_labels)

    # Source is the local temp directory. Destination is the GCS path.
    GCS_PATH = f"{GCS_BUCKET_NAME}/{GCS_DESTINATION_FOLDER}"
    print(f"\nUploading processed features to {GCS_PATH}...")

    # The -m flag runs the command multi-threaded (faster) and -r copies the directory recursively
    !gsutil -m cp -r {FEATURE_OUTPUT_DIR} {GCS_PATH}

    print("\nUpload to GCS complete. Features are ready for training notebook.")

# --- EXECUTION ---
create_and_save_features()

Processing Class: A (Label: 0)
Initializing MediaPipe Hands Detector...


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed and saved A. Freeing memory...
Processing Class: B (Label: 1)
Processed and saved B. Freeing memory...
Processing Class: C (Label: 2)
Processed and saved C. Freeing memory...
Processing Class: D (Label: 3)
Processed and saved D. Freeing memory...
Processing Class: E (Label: 4)
Processed and saved E. Freeing memory...
Processing Class: F (Label: 5)
Processed and saved F. Freeing memory...
Processing Class: G (Label: 6)
Processed and saved G. Freeing memory...
Processing Class: H (Label: 7)
Processed and saved H. Freeing memory...
Processing Class: I (Label: 8)
Processed and saved I. Freeing memory...
Processing Class: J (Label: 9)
Processed and saved J. Freeing memory...
Processing Class: K (Label: 10)
Processed and saved K. Freeing memory...
Processing Class: L (Label: 11)
Processed and saved L. Freeing memory...
Processing Class: M (Label: 12)
Processed and saved M. Freeing memory...
Processing Class: N (Label: 13)
Processed and saved N. Freeing memory...
Processing Class: O

### Final Concatenation

In [8]:
import os
print("Python CWD:", os.getcwd())

Python CWD: /content


In [ ]:
import os
import numpy as np
import gc
import shutil

REPO_NAME = 'asl-neural-app'
FEATURE_OUTPUT_DIR = os.path.join('/content/', REPO_NAME, 'processed_data', 'class_splits')
GCS_BUCKET_NAME = "gs://asl-keypoint-data-storage-2025"
GCS_DESTINATION_FOLDER = "processed_features_v1"

print("Starting memory-optimized final concatenation...")

# 1. Identify all temporary class files that need to be merged
temp_files = sorted(os.listdir(FEATURE_OUTPUT_DIR))
keypoint_files = [os.path.join(FEATURE_OUTPUT_DIR, f) for f in temp_files if f.startswith('keypoints_')]
cnn_files = [os.path.join(FEATURE_OUTPUT_DIR, f) for f in temp_files if f.startswith('cnn_')]
label_files = [os.path.join(FEATURE_OUTPUT_DIR, f) for f in temp_files if f.startswith('labels_')]

# Check if files were found
if not keypoint_files:
    raise FileNotFoundError("No temporary keypoint files found. Check FEATURE_OUTPUT_DIR path.")
if not cnn_files:
    raise FileNotFoundError("No temporary cnn files found. Check FEATURE_OUTPUT_DIR path.")
if not label_files:
    raise FileNotFoundError("No temporary label files found. Check FEATURE_OUTPUT_DIR path.")

# 2. Memory-Optimized Concatenation (Loading one-by-one and overwriting)

def merge_files_efficiently(file_list, final_name):
    """Loads files sequentially and saves the final result."""

    output_path = os.path.join(FEATURE_OUTPUT_DIR, final_name)
    print(f"Merging {len(file_list)} files into {final_name}...")

    all_arrays = [np.load(f) for f in file_list]
    merged_array = np.concatenate(all_arrays)
    np.save(output_path, merged_array)

    # Crucial: Delete objects and force garbage collection after each merge
    del all_arrays, merged_array
    gc.collect()
    print(f"Successfully saved {final_name}.")
    return output_path

# Execute the merges
final_keypoints_path = merge_files_efficiently(keypoint_files, 'X_keypoints.npy')
final_cnn_path = merge_files_efficiently(cnn_files, 'X_cnn_images.npy')
final_labels_path = merge_files_efficiently(label_files, 'y_labels.npy')

print("\nAll final feature files created successfully on local disk.")

# 3. Upload to GCS
GCS_PATH = f"{GCS_BUCKET_NAME}/{GCS_DESTINATION_FOLDER}"
print(f"Uploading final processed features from {FEATURE_OUTPUT_DIR} to {GCS_PATH}...")

print(f"Uploading final feature files to {GCS_PATH}...")

# 1. Upload X_keypoints.npy
!gsutil cp {FEATURE_OUTPUT_DIR}/X_keypoints.npy {GCS_PATH}/X_keypoints.npy

# 2. Upload X_cnn_images.npy
!gsutil cp {FEATURE_OUTPUT_DIR}/X_cnn_images.npy {GCS_PATH}/X_cnn_images.npy

# 3. Upload y_labels.npy
!gsutil cp {FEATURE_OUTPUT_DIR}/y_labels.npy {GCS_PATH}/y_labels.npy

print("\nUpload to GCS complete. Only final files were uploaded.")

print("\nUpload to GCS complete. Data processing pipeline finished! 🎉")

Starting memory-optimized final concatenation...
Merging 29 files into X_keypoints.npy...
Successfully saved X_keypoints.npy.
Merging 29 files into X_cnn_images.npy...


In [1]:
import os
import numpy as np
import gc

# --- CONFIGURATION (PLEASE VERIFY THESE PATHS) ---
# Replace 'your_repo_name' with your actual cloned repository name if using absolute path
REPO_NAME = 'asl-neural-app'
FEATURE_OUTPUT_DIR = os.path.join('/content/', REPO_NAME, 'processed_data', 'class_splits')
GCS_BUCKET_NAME = "gs://asl-keypoint-data-storage-2025"
GCS_DESTINATION_FOLDER = "processed_features_v1"
# --------------------------------------------------

print("--- Starting Memory-Mapped Merge for X_cnn_images ---")

# 1. Identify all temporary files and verify paths
try:
    temp_files = sorted(os.listdir(FEATURE_OUTPUT_DIR))
    label_files = [os.path.join(FEATURE_OUTPUT_DIR, f) for f in temp_files if f.startswith('labels_')]
    cnn_files = [os.path.join(FEATURE_OUTPUT_DIR, f) for f in temp_files if f.startswith('cnn_')]
except FileNotFoundError:
    print(f"Error: The directory {FEATURE_OUTPUT_DIR} was not found. Please check REPO_NAME.")
    exit()

if not cnn_files or not label_files:
    print("Error: No intermediate 'cnn_*.npy' or 'labels_*.npy' files found. Cannot proceed.")
    exit()

# 2. Calculate the required final shape (must be done in RAM, but only metadata)
print(f"Found {len(cnn_files)} intermediate files.")

# Calculate the total number of samples (rows)
total_samples = sum(np.load(f).shape[0] for f in label_files)

# Get the shape of a single image (e.g., (224, 224, 3))
cnn_image_shape = np.load(cnn_files[0]).shape[1:]

print(f"Total Samples to Merge: {total_samples}")
print(f"Image Feature Shape: {cnn_image_shape}")

# 3. Create and Populate the Memory-Mapped Array
FINAL_CNN_PATH = os.path.join(FEATURE_OUTPUT_DIR, 'X_cnn_images.npy')
current_row = 0

print(f"Creating memory-mapped file at: {FINAL_CNN_PATH}")

# Create the destination memory-mapped array (mode='w+' means create/write)
X_cnn_final_map = np.memmap(
    FINAL_CNN_PATH,
    dtype=np.float32,
    mode='w+',
    shape=(total_samples, *cnn_image_shape)
)

# Iteratively write data into the memory-mapped file
for i, cnn_file in enumerate(cnn_files):
    # Load one small class array into RAM
    X_cnn_class = np.load(cnn_file)
    num_samples = X_cnn_class.shape[0]

    # Write the small array directly into the correct slice of the large file on disk
    X_cnn_final_map[current_row:current_row + num_samples] = X_cnn_class

    # Update the row counter
    current_row += num_samples

    print(f"  -> Wrote file {i+1}/{len(cnn_files)} ({num_samples} samples)")

    # Crucial: Delete objects and force garbage collection after each loop
    del X_cnn_class
    gc.collect()

    # Flush ensures data is written to disk immediately
    X_cnn_final_map.flush()

print("\nStep 1 of 2: X_cnn_images successfully merged and saved locally.")

# Final cleanup of the memmap object before GCS upload
del X_cnn_final_map
gc.collect()

# 4. Upload the final file to GCS
GCS_PATH = f"{GCS_BUCKET_NAME}/{GCS_DESTINATION_FOLDER}"
GCS_DESTINATION_FILE = os.path.basename(FINAL_CNN_PATH)

print(f"\nStep 2 of 2: Uploading {GCS_DESTINATION_FILE} to {GCS_PATH}...")
# Use gsutil cp to copy the local file to the GCS path
!gsutil cp {FINAL_CNN_PATH} {GCS_PATH}/{GCS_DESTINATION_FILE}

print("\nSUCCESS: X_cnn_images.npy uploaded to GCS. Ready to move to keypoints and labels.")

--- Starting Memory-Mapped Merge for X_cnn_images ---
Found 29 intermediate files.
Total Samples to Merge: 63676
Image Feature Shape: (224, 224, 3)
Creating memory-mapped file at: /content/asl-neural-app/processed_data/class_splits/X_cnn_images.npy
  -> Wrote file 1/29 (2187 samples)
  -> Wrote file 2/29 (2207 samples)
  -> Wrote file 3/29 (1988 samples)
  -> Wrote file 4/29 (2463 samples)
  -> Wrote file 5/29 (2308 samples)
  -> Wrote file 6/29 (2876 samples)
  -> Wrote file 7/29 (2440 samples)
  -> Wrote file 8/29 (2393 samples)
  -> Wrote file 9/29 (2384 samples)
  -> Wrote file 10/29 (2578 samples)
  -> Wrote file 11/29 (2700 samples)
  -> Wrote file 12/29 (2527 samples)
  -> Wrote file 13/29 (1565 samples)
  -> Wrote file 14/29 (1276 samples)
  -> Wrote file 15/29 (2265 samples)
  -> Wrote file 16/29 (2042 samples)
  -> Wrote file 17/29 (2093 samples)
  -> Wrote file 18/29 (2541 samples)
  -> Wrote file 19/29 (2551 samples)
  -> Wrote file 20/29 (2349 samples)
  -> Wrote file 21/2

KeyboardInterrupt: 

## Data: Loading and Splitting

In [ ]:
# --- Configuration (Must match Phase 2 upload settings) ---
GCS_BUCKET_NAME = "gs://asl-keypoint-data-storage-2025"
GCS_DESTINATION_FOLDER = "processed_features_v1"
LOCAL_LOAD_DIR = 'gcs_loaded_data'
TEMP_FEATURE_DIR = os.path.join(LOCAL_LOAD_DIR, GCS_DESTINATION_FOLDER, 'temp_feature_dump')

os.makedirs(LOCAL_LOAD_DIR, exist_ok=True)

GCS_PATH = f"{GCS_BUCKET_NAME}/{GCS_DESTINATION_FOLDER}/temp_feature_dump"
print(f"Downloading processed features from {GCS_PATH}...")

# Download the files recursively from the GCS folder to the local directory
!gsutil -m cp -r {GCS_PATH} {LOCAL_LOAD_DIR}

print("Download complete. Loading arrays into memory...")

# Load the data arrays (These files were saved in the temp_feature_dump folder)
X_keypoints = np.load(os.path.join(TEMP_FEATURE_DIR, 'X_keypoints.npy'))
X_cnn_images = np.load(os.path.join(TEMP_FEATURE_DIR, 'X_cnn_images.npy'))
y_labels = np.load(os.path.join(TEMP_FEATURE_DIR, 'y_labels.npy'))

print(f"Total Samples Loaded: {X_keypoints.shape[0]}")